<a href="https://www.kaggle.com/code/kaiyoo88/kpop-idols-w2v?scriptVersionId=204099270" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt # tokenizer for Korean 

In [ ]:
train_data = pd.read_csv('youtube_comments.txt')

In [ ]:
train_data.head()

In [ ]:
print(len(train_data)) # #comments

In [ ]:
# NULL check
print(train_data.isnull().values.any())

In [ ]:
train_data = train_data.dropna(how = 'any') 
print(train_data.isnull().values.any())
print(len(train_data)) 

In [ ]:
# Only Korean comments were collected during data collection process.
# remove all other charcaters other than Hangeul.
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","", regex=True)

In [ ]:
# tokenizer for Korean comments. Use nltk or other tokenizer for English comments
okt = Okt() 

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
tokenized_data = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

In [ ]:
# Comments length distribution
print('max comment length :',max(len(comment) for comment in tokenized_data))
print('avg comment length :',sum(map(len, tokenized_data))/len(tokenized_data))
plt.hist([len(comment) for comment in tokenized_data], bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

## Train Youtube comments on idol groups with Word2Vec 

In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = tokenized_data, vector_size = 100, window = 5, min_count = 5, workers = 4, sg = 0)

In [ ]:
model.wv.save_word2vec_format('ko_w2v') # 모델 저장

In [ ]:
model.wv.vectors.shape # 100 dimension

In [ ]:
print(model.wv.most_similar("웬디"))

In [ ]:
print(model.wv.most_similar("조이"))

## Visualisation for Word Embedding

In [ ]:
# Generate 2 tsv files for embedding visualisation.
!python -m gensim.scripts.word2vec2tensor --input ko_w2v --output ko_w2v

In [ ]:
# https://projector.tensorflow.org/
# Load ko_w2v_tensor.tsv and ko_w2v_metadata.tsv 